In [ ]:
import os
import os.path as op
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import Parallel, delayed
from scipy.stats import permutation_test, ttest_rel


sns.set_style("ticks")
sns.set_context("talk", font_scale=1.2, rc={"axes.labelpad": 10})
pd.set_option("display.float_format", "{:.3}".format)

warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option("display.float_format", "{:.3}".format)

In [ ]:
# Define helper functions
def load_corr_wrapper(path, idx=None):
    if idx is not None:
        return np.load(path)[idx]
    return np.load(path)


def extract_diagonal(mat):
    n_dims = len(mat.shape)
    if n_dims == 3:
        raise ValueError("Input matrix must be 2D")
    return np.array([mat[i, i] for i in range(mat.shape[0])])


def fingerprinting_score(corr_matrix):
    """
    Computes the fingerprinting score from a correlation matrix.

    Parameters:
    corr_matrix (numpy.ndarray): A 2D numpy array containing the correlation matrix.

    Returns:
    float: The fingerprinting score.
    """
    # Initialize a list to store the fingerprinting score for each row
    fp_score = []

    # Iterate over each row in the correlation matrix
    for row_index, row in enumerate(corr_matrix):
        # Check if the maximum value in the row is on the diagonal
        is_max_on_diagonal = np.argmax(row) == row_index

        # Check if the diagonal element is the only occurrence of that value in the row
        is_unique_in_row = np.count_nonzero(row == row[row_index]) == 1

        # If both conditions are met, append True to the fp_score list, otherwise append False
        fp_score.append(is_max_on_diagonal and is_unique_in_row)

    # Calculate the fingerprinting score as the mean of the fp_score list
    return np.mean(fp_score)


def perm_ttest(a, b, num_permutations=1000, seed=42):
    def statistic(a, b):
        return ttest_rel(a, b).statistic

    res = permutation_test(
        (a, b),
        statistic,
        vectorized=False,
        permutation_type="samples",
        alternative="two-sided",
        random_state=seed,
        n_resamples=num_permutations,
    )
    return res.statistic, res.pvalue


def compute_statistics(
    df,
    contrasts,
    main_model="DeepTaskGen",
    compare_models=("Average", "Retest", "Tavor"),
):
    def run_permutation_test(cont, model):
        t_stat, p_value = perm_ttest(
            df[(df["Contrast"] == cont) & (df["Method"] == main_model)]["Corr"],
            df[(df["Contrast"] == cont) & (df["Method"] == model)]["Corr"],
        )
        return {
            "Contrast": cont,
            "Model": model,
            "t_stat": t_stat,
            "p_value": p_value,
        }

    perm_results = Parallel(n_jobs=-1)(
        delayed(run_permutation_test)(cont, model)
        for cont in contrasts
        for model in compare_models
    )
    return pd.DataFrame(perm_results)

In [ ]:
# Load HCP subjects
ABS_PATH = op.abspath(op.join(os.getcwd(), "../../.."))
TEST_SUBJ = op.join(
    ABS_PATH, "experiments/transfer_learning/hcp_development/data/hcpd_test_ids.txt"
)

RESULTS_PATH = op.join(
    ABS_PATH, "experiments/transfer_learning/hcp_development/results/figures"
)
os.makedirs(RESULTS_PATH, exist_ok=True)

INCLUDE_CONTRASTS = (
    "EMOTION FACES-SHAPES",
    "GAMBLING REWARD",
)

# DeepTaskGen – Finetuned = Red, DeepTaskGen - NoFinetune = Yellow, Tavor = Blue
PALETTE = {
    "Finetune": "#AE3033",
    "No Finetune": "#FBDF4F",
    "Linear Regression": "#283F94",
}

In [ ]:
# Train-test correlation matrices.
PRED_BY_MODEL = {
    "EMOTION\nFACES-SHAPES": {
        "Linear Regression": load_corr_wrapper(
            op.join(
                ABS_PATH,
                "experiments/transfer_learning/hcp_development/results/tavor/corr_scores_emotion_faces-shapes.npy",
            ),
        ),
        "No Finetune": load_corr_wrapper(
            op.join(
                ABS_PATH,
                "experiments/transfer_learning/hcp_development/results/nofinetune/corr_scores_emotion_faces-shapes.npy",
            ),
        ),
        "Finetune": load_corr_wrapper(
            op.join(
                ABS_PATH,
                "experiments/transfer_learning/hcp_development/results/finetuned_50_0.001_gambling-reward/corr_scores_emotion_faces-shapes.npy",
            )
        ),
    },
    "GAMBLING\nREWARD": {
        "Linear Regression": load_corr_wrapper(
            op.join(
                ABS_PATH,
                "experiments/transfer_learning/hcp_development/results/tavor/corr_scores_gambling_reward.npy",
            ),
        ),
        "No Finetune": load_corr_wrapper(
            op.join(
                ABS_PATH,
                "experiments/transfer_learning/hcp_development/results/nofinetune/corr_scores_gambling_reward.npy",
            ),
        ),
        "Finetune": load_corr_wrapper(
            op.join(
                ABS_PATH,
                "experiments/transfer_learning/hcp_development/results/finetuned_50_0.001_emotion-faces-shapes/corr_scores_gambling_reward.npy",
            ),
        ),
    },
}


# Prepare HCP Results DataFrame
def prepare_results_df():
    corr_df = pd.DataFrame()
    fp_df = pd.DataFrame()
    for cont in PRED_BY_MODEL.keys():
        for model in PRED_BY_MODEL[cont].keys():
            corr_df = pd.concat(
                [
                    corr_df,
                    pd.DataFrame(
                        extract_diagonal(np.squeeze(PRED_BY_MODEL[cont][model])).T,
                        columns=["Corr"],
                    ).assign(Method=model, Contrast=cont),
                ]
            )
            fp_df = pd.concat(
                [
                    fp_df,
                    pd.DataFrame(
                        [fingerprinting_score(np.squeeze(PRED_BY_MODEL[cont][model]))],
                        columns=["Fingerprint"],
                    ).assign(Method=model, Contrast=cont),
                ]
            )
    return corr_df, fp_df


# Prepare dataframes for reconstruction accuracy and discriminability (i.e., fingerprinting score)
corr_df, fp_df = prepare_results_df()

# Save the dataframes into .csv files.
corr_df.groupby(["Contrast", "Method"]).mean().to_csv(
    op.join(RESULTS_PATH, "recon.csv"),
    float_format="%.3f",
    decimal=",",
    sep=";",
)
fp_df.groupby(["Contrast", "Method"]).mean().to_csv(
    op.join(RESULTS_PATH, "disc.csv"),
    float_format="%.3f",
    decimal=",",
    sep=";",
)

In [ ]:
# Plot Reconstruction Accuracy
plt.figure(figsize=(14, 10), dpi=300)
sns.boxplot(data=corr_df, x="Contrast", y="Corr", hue="Method", palette=PALETTE)
plt.ylim(-0.1, 0.75)  # Set y-axis limits
plt.ylabel("Reconstruction")
plt.legend()
sns.despine(offset=10, trim=True)
plt.savefig(op.join(RESULTS_PATH, "recon.svg"), bbox_inches="tight")
plt.show()

# Plot Fingerprinting Score
plt.figure(figsize=(14, 10), dpi=300)
sns.pointplot(data=fp_df, x="Contrast", y="Fingerprint", hue="Method", palette=PALETTE)
plt.ylim(0, 1)  # Set y-axis limits
sns.despine(offset=10, trim=True)
plt.legend()
plt.ylabel("Discriminbility")
plt.subplots_adjust(bottom=0.2)
plt.savefig(op.join(RESULTS_PATH, "disc.svg"), bbox_inches="tight")
plt.show()

In [ ]:
## Post-hoc Comparisons between models in terms of reconstruction accuracy and discriminability
# Significance is determined using permutation tests with 1000 permutations.
target_contrasts = ["EMOTION\nFACES-SHAPES", "GAMBLING\nREWARD"]
target_models = ("Linear Regression", "No Finetune")

# RECONSTRUCTION ACCURACY
print("Reconstruction Accuracy")
recon_ttest = compute_statistics(
    corr_df,
    target_contrasts,
    main_model="Finetune",
    compare_models=target_models,
)
recon_ttest.to_csv(
    op.join(RESULTS_PATH, "recon_ttest.csv"),
    float_format="%.3f",
    index=False,
    decimal=",",
    sep=";",
)
print(recon_ttest)

# DISCRIMINABILITY SCORE
print("\nDiscriminability")
print(
    pd.read_csv(
        op.join(RESULTS_PATH, "disc.csv"),
        sep=";",
        decimal=",",
        index_col=None,
    )
)